# Installing and importing Dependencies

In [14]:
!pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable


In [15]:
!pip install tensorflow-io

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import os
import matplotlib.pyplot as plt
import tensorflow as plt
import tensorflow_io as tfio

# Data Loading

In [ ]:
!pip install jupyter_contrib_nbextensions